# 4 prepare remote deploy

## prepare turtlebot3
You can use either turtlebot3_simulator (A.) or actual turtlebot3 robot (B.).

### A. use "turtlebot3 simulator" on Ubuntu desktop

#### prepare Ubuntu desktop as a turlebot3 simulator 

1. prepare Ubuntu 16.04 desktop
    * If you use a virtual machine on VirtualBox as Ubuntu desktop, confirm below:
        * "3d acceralation" of display is "OFF"
        * set `export LIBGL_ALWAYS_SOFTWARE=1` to your .bashrc
1. set `export TURTLEBOT3_MODEL=waffle` to your .bashrc
1. install `ros-kinetic-desktop-full` and `ros-kinetic-rqt-*` using `apt`.
1. create ROS workspace.
1. clone repositories of turtlebot3_simulator from github.
    * https://github.com/ROBOTIS-GIT/turtlebot3.git
    * https://github.com/ROBOTIS-GIT/turtlebot3_msgs.git
    * https://github.com/ROBOTIS-GIT/turtlebot3_simulations.git
1. make repositories using `catkin_make`.

### B. use actual "turtlebot3" robot

#### prepare Ubuntu desktop as a turlebot3 simulator

1. prepare turtlebot3
1. make repositories using `catkin_make`.

----

change ${CORE_ROOT} to your path of `core`.

In [ ]:
export CORE_ROOT="${HOME}/core"

change ${PJ_ROOT} to your path of `example-turtlebot3`.

In [ ]:
export PJ_ROOT="${HOME}/example-turtlebot3"
cd ${PJ_ROOT};pwd

example)
```
/Users/user/example-turtlebot3
```

## load environment variables

load from `core`

In [ ]:
source ${CORE_ROOT}/docs/environments/minikube/env

load from `example-turtlebot3`

In [ ]:
source ${PJ_ROOT}/docs/environments/minikube/env

## set external ip addr of minikube machine

#### for macOS

In [ ]:
export IFNAME="en0"
export EXTERNAL_HOST_IPADDR=$(ifconfig ${IFNAME} | awk '/inet / {print $2}');echo ${EXTERNAL_HOST_IPADDR}

#### for Ubuntu

In [ ]:
export IFNAME="enp0s25"
export EXTERNAL_HOST_IPADDR=$(ifconfig ${IFNAME} | awk '/inet / {print $2}' | cut -d: -f2);echo ${EXTERNAL_HOST_IPADDR}

## setup docker

1. install required packages

```bash
turtlebot3@turtlebot3:~$ sudo apt update
turtlebot3@turtlebot3:~$ sudo apt upgrade -y
turtlebot3@turtlebot3:~$ sudo apt install apt-transport-https ca-certificates curl gnupg-agent software-properties-common
  ```
2. add Docker’s official GPG key:

```bash
turtlebot3@turtlebot3:~$ curl -fsSL https://download.docker.com/linux/ubuntu/gpg | sudo apt-key add -
```
3. set up the stable repository of Docker

```bash
turtlebot3@turtlebot3:~$ sudo add-apt-repository "deb [arch=amd64] https://download.docker.com/linux/ubuntu $(lsb_release -cs) stable"
```
4. install Docker

```bash
turtlebot3@turtlebot3:~$ sudo apt update
turtlebot3@turtlebot3:~$ sudo apt-get install -y docker-ce docker-ce-cli containerd.io
```
5. verify instration

```bash
turtlebot3@turtlebot3:~$ sudo docker version
turtlebot3@turtlebot3:~$ sudo docker run hello-world
```

### add inserucre registry

In [ ]:
echo "sudo mkdir -p /etc/systemd/system/docker.service.d/; cat << __EOS__ | sudo tee /etc/systemd/system/docker.service.d/override.conf
[Service]
ExecStart=
ExecStart=/usr/bin/dockerd -H fd:// --insecure-registry=${EXTERNAL_HOST_IPADDR}:5000
__EOS__"

1. add insecure registry to turtlebot3's docker using above command
2. restart docker daemon

```bash
turtlebot3@turtlebot3-fake:~$ sudo systemctl daemon-reload
turtlebot3@turtlebot3-fake:~$ sudo systemctl restart docker.service
turtlebot3@turtlebot3-fake:~$ sudo systemctl status docker.service
```

## setup minikube

1. setup minikube

```bash
turtlebot3@turtlebot3:~$ curl -Lo minikube https://storage.googleapis.com/minikube/releases/v1.0.0/minikube-linux-amd64 && chmod +x minikube && sudo cp minikube /usr/local/bin/ && rm minikube
```
2. verify instration

```bash
turtlebot3@turtlebot3:~$ minikube version
```

## setup kubectl

1. setup kubectl

```bash
turtlebot3@turtlebot3:~$ curl -LO https://storage.googleapis.com/kubernetes-release/release/v1.14.1/bin/linux/amd64/kubectl && chmod +x kubectl && sudo cp kubectl /usr/local/bin/ && rm kubectl
```
2. verify instration

```bash
turtlebot3@turtlebot3:~$ kubectl version --client
```

## start minikube without virtualization

### generate a command to add Insecure Repository

#### for macOS

In [ ]:
NWNAME=$(VBoxManage showvminfo ${MINIKUBE_NAME} | grep "Host-only Interface" | awk 'match($0, /vboxnet[0-9]+/){print substr($0,RSTART,RLENGTH)}')
HOST_IPADDR=$(ifconfig ${NWNAME} | awk '/inet / {print $2}')
NETMASK_HEX=$(ifconfig ${NWNAME} | awk '/netmask / {print $4}')
NETMASK=$(echo "${NETMASK_HEX:2}" | perl -pe '$_ = unpack("B32", pack("H*", $_)); s/0+$//g; $_ = length')
echo 'cat ${HOME}/.minikube/machines/minikube/config.json | perl -pse '"'"'s/"InsecureRegistry": \[/"InsecureRegistry": [\n                "$h\/$m",/g;'"' -- -h=${EXTERNAL_HOST_IPADDR} -m=${NETMASK}"' > /tmp/config.json;mv /tmp/config.json ${HOME}/.minikube/machines/minikube/config.json'

#### for Ubuntu

In [ ]:
NWNAME=$(VBoxManage showvminfo ${MINIKUBE_NAME} | grep "Host-only Interface" | awk 'match($0, /vboxnet[0-9]+/){print substr($0,RSTART,RLENGTH)}')
HOST_IPADDR=$(ifconfig ${NWNAME}  | awk '/inet / {print $2}' | cut -d: -f2)
NETMASK_IP=$(ifconfig ${NWNAME} | awk '/Mask/ {print $4}' | cut -d: -f2)
NETMASK=$(ipcalc ${HOST_IPADDR} ${NETMASK_IP} | awk '/Netmask: / {print $4}')
echo 'cat ${HOME}/.minikube/machines/minikube/config.json | perl -pse '"'"'s/"InsecureRegistry": \[/"InsecureRegistry": [\n                "$h\/$m",/g;'"' -- -h=${EXTERNAL_HOST_IPADDR} -m=${NETMASK}"' > /tmp/config.json;mv /tmp/config.json ${HOME}/.minikube/machines/minikube/config.json'

### start minikube

1. delete minikube if already started

```bash
turtlebot3@turtlebot3:~$ sudo minikube stop
turtlebot3@turtlebot3:~$ sudo minikube delete
turtlebot3@turtlebot3:~$ sudo rm -rf /etc/kubernetes/
turtlebot3@turtlebot3:~$ sudo rm -rf $HOME/.minikube/
turtlebot3@turtlebot3:~$ rm -rf $HOME/.kube/
```
2. setup environment variables

```bash
turtlebot3@turtlebot3:~$ export MINIKUBE_WANTUPDATENOTIFICATION=false
turtlebot3@turtlebot3:~$ export MINIKUBE_WANTREPORTERRORPROMPT=false
turtlebot3@turtlebot3:~$ export MINIKUBE_HOME=$HOME
turtlebot3@turtlebot3:~$ export CHANGE_MINIKUBE_NONE_USER=true
turtlebot3@turtlebot3:~$ export KUBECONFIG=$HOME/.kube/config
```

```bash
turtlebot3@turtlebot3:~$ export CPU_CORE_NUM="1"
turtlebot3@turtlebot3:~$ export MEMORY_MB=2048
turtlebot3@turtlebot3:~$ export K8S_VERSION="v1.13.5"
```
3. create directories and `.kube/config`

```
turtlebot3@turtlebot3:~$ mkdir -p $HOME/.kube $HOME/.minikube
turtlebot3@turtlebot3:~$ touch $KUBECONFIG
```
4. start minikube without virtualization

```bash
turtlebot3@turtlebot3:~$ sudo -E minikube start --cpus ${CPU_CORE_NUM} --memory ${MEMORY_MB} --vm-driver=none --kubernetes-version ${K8S_VERSION} --feature-gates=CoreDNS=false
```
5. rewite `${HOME}/.minikube/machines/minikube/config.json` using above command
6. restart minikube

```bash
turtlebot3@turtlebot3-fake:~$ minikube stop
turtlebot3@turtlebot3:~$ sudo -E minikube start --cpus ${CPU_CORE_NUM} --memory ${MEMORY_MB} --vm-driver=none --kubernetes-version ${K8S_VERSION} --feature-gates=CoreDNS=false
```
7. verify minikube

```bash
turtlebot3@turtlebot3:~$ kubectl version
turtlebot3@turtlebot3:~$ kubectl get nodes
```
8. confirm that all PODs are `Running`

```bash
turtlebot3@turtlebot3:~$ kubectl get pods --all-namespaces
```
```bash
example)
NAMESPACE     NAME                               READY   STATUS    RESTARTS   AGE
kube-system   etcd-minikube                      1/1     Running   0          3m26s
kube-system   kube-addon-manager-minikube        1/1     Running   0          3m25s
kube-system   kube-apiserver-minikube            1/1     Running   0          3m17s
kube-system   kube-controller-manager-minikube   1/1     Running   0          3m15s
kube-system   kube-dns-86b8794d97-gwqkr          3/3     Running   0          4m8s
kube-system   kube-proxy-f4pvl                   1/1     Running   0          4m8s
kube-system   kube-scheduler-minikube            1/1     Running   0          3m14s
kube-system   storage-provisioner                1/1     Running   0          4m6s
```

### confirm the dns settings of minkube

1. try to dig `www.google.com`

```bash
turtlebot3@turtlebot3:~$ kubectl run -it --rm --restart=Never dig --image tutum/dnsutils -- dig www.google.com
```
2. if you encountered `connection timed out`, you have to setup the additional nameservers to `kube-dns`

### setup additional nameservers to `kube-dns`

1. prepare `/tmp/kube-dns-configmap.yaml` by using below command:

```bash
turtlebot3@turtlebot3:~$ cat << __EOF__ > /tmp/kube-dns-configmap.yaml
apiVersion: v1
kind: ConfigMap
metadata:
  name: kube-dns
  namespace: kube-system
  labels:
    addonmanager.kubernetes.io/mode: EnsureExists
data:
  upstreamNameservers: |-
    ["8.8.8.8", "8.8.4.4"]
__EOF__
```
2. apply `/tmp/kube-dns-configmap.yaml`

```bash
turtlebot3@turtlebot3:~$ kubectl apply -f /tmp/kube-dns-configmap.yaml
```
3. delete the pod of `kube-dns` (restart `kube-dns` automatically)

```bash
turtlebot3@turtlebot3:~$ kubectl delete pod -n kube-system $(kubectl get pods -n kube-system -l k8s-app=kube-dns -o template --template "{{(index .items 0).metadata.name}}")
```
4. confirm that `kube-dns` is started

```bash
turtlebot3@turtlebot3:~$ kubectl get pods -n kube-system -l k8s-app=kube-dns
NAME                        READY     STATUS    RESTARTS   AGE
kube-dns-86f4d74b45-x7m75   3/3       Running   0          2m
```
5. retry to dig `www.google.com`

```bash
turtlebot3@turtlebot3:~$ kubectl run -it --rm --restart=Never dig --image tutum/dnsutils -- dig www.google.com
```

## register deployer service

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-ServicePath: ${DEPLOYER_SERVICEPATH}" -H "Content-Type: application/json" http://${HOST_IPADDR}:8080/idas/ul20/manage/iot/services/ -X POST -d @- <<__EOS__
{
  "services": [
    {
      "apikey": "${DEPLOYER_TYPE}",
      "cbroker": "http://orion:1026",
      "resource": "/iot/d",
      "entity_type": "${DEPLOYER_TYPE}"
    }
  ]
}
__EOS__

expected)
```json
{}
```

### confirm registered service

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DEPLOYER_SERVICEPATH}" http://${HOST_IPADDR}:8080/idas/ul20/manage/iot/services/ | jq .

example)
```bash
{
  "count": 1,
  "services": [
    {
      "_id": "5b90ba6c0c4a497125314e7c",
      "subservice": "/deployer",
      "service": "fiwaredemo",
      "apikey": "deployer",
      "resource": "/iot/d",
      "__v": 0,
      "attributes": [],
      "lazy": [],
      "commands": [],
      "entity_type": "deployer",
      "internal_attributes": [],
      "static_attributes": []
    }
  ]
}
```

## register deployer device

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-ServicePath: ${DEPLOYER_SERVICEPATH}" -H "Content-Type: application/json" http://${HOST_IPADDR}:8080/idas/ul20/manage/iot/devices/ -X POST -d @- <<__EOS__
{
  "devices": [
    {
      "device_id": "${DEPLOYER_ID}",
      "entity_name": "${DEPLOYER_ID}",
      "entity_type": "${DEPLOYER_TYPE}",
      "timezone": "Asia/Tokyo",
      "protocol": "UL20",
      "attributes": [
        {
          "name": "deployment",
          "type": "string"
        },
        {
          "name": "label",
          "type": "string"
        },
        {
          "name": "desired",
          "type": "integer"
        },
        {
          "name": "current",
          "type": "integer"
        },
        {
          "name": "updated",
          "type": "integer"
        },
        {
          "name": "ready",
          "type": "integer"
        },
        {
          "name": "unavailable",
          "type": "integer"
        },
        {
          "name": "available",
          "type": "integer"
        }
      ],
      "commands": [
        {
          "name": "apply",
          "type": "string"
        }, {
          "name": "delete",
          "type": "string"
        }
      ],
      "transport": "AMQP"
    }
  ]
}
__EOS__

expected)
```json
{}
```

### confirm registered device

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DEPLOYER_SERVICEPATH}" http://${HOST_IPADDR}:8080/idas/ul20/manage/iot/devices/${DEPLOYER_ID}/ | jq .

example)
```bash
{
  "device_id": "deployer_01",
  "service": "fiwaredemo",
  "service_path": "/deployer",
  "entity_name": "deployer_01",
  "entity_type": "deployer",
  "transport": "AMQP",
  "attributes": [
    {
      "object_id": "deployment",
      "name": "deployment",
      "type": "string"
    },
    {
      "object_id": "label",
      "name": "label",
      "type": "string"
    },
    {
      "object_id": "desired",
      "name": "desired",
      "type": "integer"
    },
    {
      "object_id": "current",
      "name": "current",
      "type": "integer"
    },
    {
      "object_id": "updated",
      "name": "updated",
      "type": "integer"
    },
    {
      "object_id": "ready",
      "name": "ready",
      "type": "integer"
    },
    {
      "object_id": "unavailable",
      "name": "unavailable",
      "type": "integer"
    },
    {
      "object_id": "available",
      "name": "available",
      "type": "integer"
    }
  ],
  "lazy": [],
  "commands": [
    {
      "object_id": "apply",
      "name": "apply",
      "type": "string"
    },
    {
      "object_id": "delete",
      "name": "delete",
      "type": "string"
    }
  ],
  "static_attributes": [],
  "protocol": "UL20"
}
```

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DEPLOYER_SERVICEPATH}" http://${HOST_IPADDR}:8080/orion/v2/entities/${DEPLOYER_ID}/ | jq .

example)
```bash
{
  "id": "deployer_01",
  "type": "deployer",
  "TimeInstant": {
    "type": "ISO8601",
    "value": " ",
    "metadata": {}
  },
  "apply_info": {
    "type": "commandResult",
    "value": " ",
    "metadata": {}
  },
  "apply_status": {
    "type": "commandStatus",
    "value": "UNKNOWN",
    "metadata": {}
  },
  "available": {
    "type": "integer",
    "value": " ",
    "metadata": {}
  },
  "current": {
    "type": "integer",
    "value": " ",
    "metadata": {}
  },
  "delete_info": {
    "type": "commandResult",
    "value": " ",
    "metadata": {}
  },
  "delete_status": {
    "type": "commandStatus",
    "value": "UNKNOWN",
    "metadata": {}
  },
  "deployment": {
    "type": "string",
    "value": " ",
    "metadata": {}
  },
  "desired": {
    "type": "integer",
    "value": " ",
    "metadata": {}
  },
  "label": {
    "type": "string",
    "value": " ",
    "metadata": {}
  },
  "ready": {
    "type": "integer",
    "value": " ",
    "metadata": {}
  },
  "unavailable": {
    "type": "integer",
    "value": " ",
    "metadata": {}
  },
  "updated": {
    "type": "integer",
    "value": " ",
    "metadata": {}
  },
  "apply": {
    "type": "string",
    "value": "",
    "metadata": {}
  },
  "delete": {
    "type": "string",
    "value": "",
    "metadata": {}
  }
}
```

## deploy `deployer` to turtlebot3

### prepare a command to create Secret of username & password

In [ ]:
echo "kubectl create secret generic mqtt-username-password --from-literal=mqtt_username=ros --from-literal=mqtt_password=${MQTT__ros}"

1. create Secret of username & password of MQTT Broker using above command

### prepare a command to create ConfigMap of mqtt endpoint

In [ ]:
echo "kubectl create configmap mqtt-config --from-literal=mqtt_use_tls=false --from-literal=mqtt_host=${EXTERNAL_HOST_IPADDR} --from-literal=mqtt_port=1883 --from-literal=device_type=${DEPLOYER_TYPE} --from-literal=device_id=${DEPLOYER_ID}"

1. create ConfigMap of host & port & topic to connect MQTT Broker using above command

### start `deployer` to operate k8s Resources through MQTT

1. prepare `/tmp/mqtt-kube-operator.yaml` using below command:

```bash
turtlebot3@turtlebot3:~$ cat << __EOF__ > /tmp/mqtt-kube-operator.yaml
apiVersion: v1
kind: ServiceAccount
metadata:
  name: mqtt-kube-operator
---
apiVersion: rbac.authorization.k8s.io/v1
kind: Role
metadata:
  name: mqtt-kube-operator
  namespace: default
rules:
- apiGroups: [""]
  resources: ["services", "configmaps", "secrets"]
  verbs: ["get", "list", "create", "update", "delete"]
- apiGroups: ["apps"]
  resources: ["deployments"]
  verbs: ["get", "list", "create", "update", "delete"]
---
apiVersion: rbac.authorization.k8s.io/v1
kind: RoleBinding
metadata:
  name: mqtt-kube-operator
  namespace: default
roleRef:
  apiGroup: rbac.authorization.k8s.io
  kind: Role
  name: mqtt-kube-operator
subjects:
- kind: ServiceAccount
  name: mqtt-kube-operator
  namespace: default
---
apiVersion: apps/v1
kind: Deployment
metadata:
  name: mqtt-kube-operator
spec:
  replicas: 1
  selector:
    matchLabels:
      app: mqtt-kube-operator
  template:
    metadata:
      labels:
        app: mqtt-kube-operator
    spec:
      serviceAccountName: mqtt-kube-operator
      containers:
      - name: mqtt-kube-operator
        image: roboticbase/mqtt-kube-operator:0.2.0
        imagePullPolicy: Always
        env:
        - name: LOG_LEVEL
          value: "info"
        - name: MQTT_USERNAME
          valueFrom:
            secretKeyRef:
              name: mqtt-username-password
              key: mqtt_username
        - name: MQTT_PASSWORD
          valueFrom:
            secretKeyRef:
              name: mqtt-username-password
              key: mqtt_password
        - name: MQTT_USE_TLS
          valueFrom:
            configMapKeyRef:
              name: mqtt-config
              key: mqtt_use_tls
        - name: MQTT_HOST
          valueFrom:
            configMapKeyRef:
              name: mqtt-config
              key: mqtt_host
        - name: MQTT_PORT
          valueFrom:
            configMapKeyRef:
              name: mqtt-config
              key: mqtt_port
        - name: DEVICE_TYPE
          valueFrom:
            configMapKeyRef:
              name: mqtt-config
              key: device_type
        - name: DEVICE_ID
          valueFrom:
            configMapKeyRef:
              name: mqtt-config
              key: device_id
        - name: REPORT_INTERVAL_SEC
          value: "1"
        - name: USE_DEPLOYMENT_STATE_REPORTER
          value: "true"
        - name: REPORT_TARGET_LABEL_KEY
          value: "report"
__EOF__
```
2. apply `/tmp/mqtt-kube-operator.yaml`

```bash
turtlebot3@turtlebot3:~$ kubectl apply -f /tmp/mqtt-kube-operator.yaml
```
3. confirm that `mqtt-kube-operator` connect to MQTT Broker

```bash
turtlebot3@turtlebot3:~$ kubectl logs -f $(kubectl get pods -l app=mqtt-kube-operator -o template --template "{{(index .items 0).metadata.name}}")
```

## test publishing the `apply` command of `deployer`

### prepare a command to subscribe all topics

In [ ]:
echo "mosquitto_sub -h ${HOST_IPADDR} -p 1883 -d -u iotagent -P ${MQTT__iotagent} -t /#"

### subscribe all topics

_Outside of this notebook_
1. open a ternminal.
1. run the above command displayed `prepare a command to subscribe all topics`.

### send a NGSI to emulate sending command

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -i -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DEPLOYER_SERVICEPATH}" -H "Content-Type: application/json" http://${HOST_IPADDR}:8080/orion/v2/entities/${DEPLOYER_ID}/attrs?type=${DEPLOYER_TYPE} -X PATCH -d @-<<__EOS__
{
  "apply": {
    "value": "{}"
  }
}
__EOS__

example)
```
HTTP/1.1 204 No Content
content-length: 0
fiware-correlator: b7bfd370-7bb5-11e9-b441-0242ac110015
date: Tue, 21 May 2019 10:46:41 GMT
x-envoy-upstream-service-time: 27
server: envoy
```

### confirm the topic

when executing the above command, show below messages on the opened terminal.

example)
```
Client mosqsub|70811-MacBook-P received PUBLISH (d0, q0, r0, m0, '/deployer/deployer_01/cmd', ... (20 bytes))
deployer_01@apply|{}
Client mosqsub|70811-MacBook-P received PUBLISH (d0, q0, r0, m0, '/deployer/deployer_01/cmdexe', ... (50 bytes))
deployer_01@apply|ignore format, skip this message
```

### confirm `deployer` log

example)

```
2019-04-24T06:16:05.808Z	INFO	handlers/messageHandler.go:110	received message: deployer_01@apply|{}
2019-04-24T06:16:05.809Z	INFO	handlers/messageHandler.go:138	data: {}
2019-04-24T06:16:05.809Z	INFO	handlers/messageHandler.go:169	invalid format, skip this message: Object 'Kind' is missing in '{}'
2019-04-24T06:16:06.309Z	INFO	handlers/messageHandler.go:105	send message: deployer_01@apply|invalid format, skip this message
```

### confirm deployer entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DEPLOYER_SERVICEPATH}" http://${HOST_IPADDR}:8080/orion/v2/entities/${DEPLOYER_ID}/ | jq .

example)
```bash
{
  "id": "deployer_01",
  "type": "deployer",
  "TimeInstant": {
    "type": "ISO8601",
    "value": "2018-09-29T04:30:22.00Z",
    "metadata": {}
  },
  "apply_info": {
    "type": "commandResult",
    "value": "invalid format, skip this message",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2018-09-29T04:29:28.956Z"
      }
    }
  },
  "apply_status": {
    "type": "commandStatus",
    "value": "OK",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2018-09-29T04:29:28.956Z"
      }
    }
  },
  "available": {
    "type": "integer",
    "value": "1",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2018-09-29T04:30:22Z"
      }
    }
  },
  "current": {
    "type": "integer",
    "value": "1",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2018-09-29T04:30:22Z"
      }
    }
  },
  "delete_info": {
    "type": "commandResult",
    "value": " ",
    "metadata": {}
  },
  "delete_status": {
    "type": "commandStatus",
    "value": "UNKNOWN",
    "metadata": {}
  },
  "deployment": {
    "type": "string",
    "value": "mqtt-kube-operator",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2018-09-29T04:30:22Z"
      }
    }
  },
  "desired": {
    "type": "integer",
    "value": "1",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2018-09-29T04:30:22Z"
      }
    }
  },
  "ready": {
    "type": "integer",
    "value": "1",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2018-09-29T04:30:22Z"
      }
    }
  },
  "unavailable": {
    "type": "integer",
    "value": "0",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2018-09-29T04:30:22Z"
      }
    }
  },
  "updated": {
    "type": "integer",
    "value": "1",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2018-09-29T04:30:22Z"
      }
    }
  },
  "apply": {
    "type": "string",
    "value": "",
    "metadata": {}
  },
  "delete": {
    "type": "string",
    "value": "",
    "metadata": {}
  }
}
```

## register `cygnus-elasticsearch` as a subscriber of deployer device

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -i -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DEPLOYER_SERVICEPATH}"  -H "Content-Type: application/json" http://${HOST_IPADDR}:8080/orion/v2/subscriptions/ -X POST -d @- <<__EOS__
{
  "subject": {
    "entities": [{
      "idPattern": "${DEPLOYER_ID}.*",
      "type": "${DEPLOYER_TYPE}"
    }],
    "condition": {
      "attrs": ["deployment", "label", "desired", "current", "updated", "ready", "unavailable", "available"]
    }
  },
  "notification": {
    "http": {
      "url": "http://cygnus-elasticsearch:5050/notify"
    },
    "attrs": ["deployment", "label", "desired", "current", "updated", "ready", "unavailable", "available"],
    "attrsFormat": "legacy"
  }
}
__EOS__

example)
```
HTTP/1.1 201 Created
content-length: 0
location: /v2/subscriptions/5baf010d9970a6a6642afb82
fiware-correlator: 159fb0aa-c3a1-11e8-bf68-82ea6d7e02e9
date: Sat, 29 Sep 2018 04:35:25 GMT
x-envoy-upstream-service-time: 4
server: envoy
```

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-ServicePath: ${DEPLOYER_SERVICEPATH}" http://${HOST_IPADDR}:8080/orion/v2/subscriptions/ | jq .

example)
```json
[
  {
    "id": "5bb054b40af338d2b9bff606",
    "status": "active",
    "subject": {
      "entities": [
        {
          "idPattern": "deployer_01.*",
          "type": "deployer"
        }
      ],
      "condition": {
        "attrs": [
          "deployment",
          "label",
          "desired",
          "current",
          "updated",
          "ready",
          "unavailable",
          "available"
        ]
      }
    },
    "notification": {
      "timesSent": 1,
      "lastNotification": "2018-09-30T04:44:36.00Z",
      "attrs": [
        "deployment",
        "label",
        "desired",
        "current",
        "updated",
        "ready",
        "unavailable",
        "available"
      ],
      "attrsFormat": "legacy",
      "http": {
        "url": "http://cygnus-elasticsearch:5050/notify"
      }
    }
  }
]
```

## confirm cygnus-elasticsearch

### set up an index of Kibana

In [ ]:
echo 'kubectl --namespace monitoring port-forward $(kubectl get pod -l k8s-app=kibana-logging --namespace monitoring -o template --template "{{(index .items 0).metadata.name}}") 5601:5601'

#### for macOS

In [ ]:
open http://localhost:5601

#### for Ubuntu

In [ ]:
xdg-open http://localhost:5601

1. show `Management -> Index Patterns`
2. set `cygnus-fiwaredemo-deployer-*` as Index Pattern, and push `Next step`
3. set `recvTime` as Time Filter field name, and push `Create index pattern`

**Index name is like: `<<IndexPrefix of cygnus>>-<<FIWARE_SERVICE>>-<<FIWARE_SERVICEPATH of depoloyer>>-<<yyyy.mm.dd>>`**

### set up a datasource of grafana

In [ ]:
echo 'kubectl --namespace monitoring port-forward $(kubectl get pod --namespace monitoring -l app=kp-grafana -o template --template "{{(index .items 0).metadata.name}}") 3000:3000'

#### for macOS

In [ ]:
open http://localhost:3000

#### for Ubuntu

In [ ]:
xdg-open http://localhost:3000

1. add a new Data Source (Elasticsearch)
    * Name: `cygnus-fiwaredemo-deployer`
    * URL: `http://elasticsearch-logging:9200`
    * Access: `Server(Default)`
    * Index name: `cygnus-fiwaredemo-deployer-*`
    * Time field name: `recvTime`
    * Version: `6.0+`
    
**Index name is like: `<<IndexPrefix of cygnus>>-<<FIWARE_SERVICE>>-<<FIWARE_SERVICEPATH of depoloyer>>-<<yyyy.mm.dd>>`**

2. import `monitoring/dashboard_turtlebot3.json`